<a href="https://colab.research.google.com/github/Balrog16/Test/blob/master/MIMII_Fan_Training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Background**

Fan audio data from MIMII database is played after combining a few files randomly to form two audio records - normal and abnormal, each lasting for 520 sec. This data is played on JBL Go speaker with SITRANS-MS placed in the middle of the speaker. This arrangement is to vibrate the sensor and obtain the stream via BLE using MATLAB scripts. There are two sets of JSON files each for Normal audio and abnormal audio data.


In [8]:
# all imports
import json
import numpy as np
import tensorflow as tf
from numpy import savetxt

In [9]:
# code to shuffle 
# due credits to Josh Bleecher Snyder
# https://stackoverflow.com/questions/4601373/better-way-to-shuffle-two-numpy-arrays-in-unison
def shuffle_in_unison_scary(a, b):
    rng_state = np.random.get_state()
    np.random.shuffle(a)
    np.random.set_state(rng_state)
    np.random.shuffle(b)

In [10]:
class SMSData:
    def __init__(self, fileName, label):
        self.fileName = fileName
        f = open(self.fileName)
        data = json.load(f)
        self.accelX = np.array(data['values']['5FF8'])
        self.accelY = np.array(data['values']['5FF9'])
        self.accelZ = np.array(data['values']['5FFA'])
        # Collapse the last two columns to one by 2-norm of the components
        self.accelX = np.sqrt(np.square(self.accelX[:,:,0]) + np.square(self.accelX[:, :, 1]))
        self.accelY = np.sqrt(np.square(self.accelY[:,:,0]) + np.square(self.accelY[:, :, 1]))
        self.accelZ = np.sqrt(np.square(self.accelZ[:,:,0]) + np.square(self.accelZ[:, :, 1]))
        # Close the file
        f.close()
        self.minCols = (min(self.accelX.shape[1], self.accelY.shape[1], self.accelZ.shape[1]))
        self.accelX  = np.transpose(self.accelX[:, 0: self.minCols])
        self.accelY  = np.transpose(self.accelY[:, 0: self.minCols])
        self.accelZ  = np.transpose(self.accelZ[:, 0: self.minCols])
        self.yData   = np.ones([self.accelY.shape[0], 1])*label
        self.featVecA = self.accelX + self.accelY + self.accelZ 
        self.featVecB  = np.dstack((self.accelX, self.accelY, self.accelZ))


In [14]:
# create empty objects
SMSDataObjNormal_1   = []
SMSDataObjNormal_2   = []
SMSDataObjabNormal_1 = []
SMSDataObjabNormal_2 = []

# define a class object to hold the data
SMSDataObjNormal_1 = SMSData("/content/sample_data/normal_1_6dB_Fan.json", 0)
SMSDataObjNormal_2 = SMSData("/content/sample_data/normal_2_6dB_Fan.json", 0)
SMSDataObjabNormal_1 = SMSData("/content/sample_data/abnormal_1_6dB_Fan.json", 1)
SMSDataObjabNormal_2 = SMSData("/content/sample_data/abnormal_2_6dB_Fan.json", 1)


In [18]:
# for debug purpose
print('No of columns used', SMSDataObjabNormal_1.minCols)
print('Size of each axis', np.shape(SMSDataObjabNormal_1.accelX))
print('Feature vector B looks like', np.shape(SMSDataObjabNormal_1.featVecB))
minCols = min(SMSDataObjNormal_1.minCols, SMSDataObjNormal_2.minCols, SMSDataObjabNormal_1.minCols, SMSDataObjabNormal_1.minCols)
print('Minimum cols across the data is', minCols)

No of columns used 513
Size of each axis (513, 256)
Feature vector B looks like (513, 256, 3)
Minimum cols across the data is 512


In [19]:
# get data in x (input) and y (output)
x = np.concatenate((SMSDataObjNormal_1.featVecB, SMSDataObjabNormal_1.featVecB, 
                    SMSDataObjNormal_2.featVecB, SMSDataObjabNormal_2.featVecB))

y = np.concatenate((SMSDataObjNormal_1.yData, SMSDataObjabNormal_1.yData, 
                    SMSDataObjNormal_2.yData, SMSDataObjabNormal_2.yData))

# now shuffle the data, it doesn't matter anymore if these appear in order 
shuffle_in_unison_scary(x, y)

print(np.shape(x), np.shape(y))

(2050, 256, 3) (2050, 1)


In [23]:
trainIdx = 1801
testIdx = 1801
endIdx = 2040
# generate a dataset for training and testing
x_train = x[1:trainIdx, :, :]
y_train = y[1:trainIdx, :]

x_test = x[testIdx:endIdx, :, :]
y_test = y[testIdx:endIdx, :]

# check vector shape
print(x_train.shape)
print(y_train.shape)

print(x_test.shape)
print(y_test.shape)

(1800, 256, 3)
(1800, 1)
(239, 256, 3)
(239, 1)


In [111]:
# check vector shape
print(x_train.shape)
print(y_train.shape)

print(x_test.shape)
print(y_test.shape)


(1800, 256, 3)
(1800, 1)
(244, 256, 3)
(244, 1)


In [24]:
# time to try NN

# model
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(256, 3)),
  tf.keras.layers.Dense(64, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(2)
])

# loss function
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

# settings
model.compile(optimizer='adam',
              loss=loss_fn,
              metrics=['accuracy'])

In [25]:
# train the dog
model.fit(x_train, y_train, epochs=25)

Epoch 1/25
57/57 [==============================] - 0s 1ms/step - loss: 6.0081 - accuracy: 0.5367
Epoch 2/25
57/57 [==============================] - 0s 1ms/step - loss: 2.5566 - accuracy: 0.6539
Epoch 3/25
57/57 [==============================] - 0s 1ms/step - loss: 1.0900 - accuracy: 0.7789
Epoch 4/25
57/57 [==============================] - 0s 1ms/step - loss: 0.4865 - accuracy: 0.8594
Epoch 5/25
57/57 [==============================] - 0s 1ms/step - loss: 0.2848 - accuracy: 0.9056
Epoch 6/25
57/57 [==============================] - 0s 1ms/step - loss: 0.1905 - accuracy: 0.9211
Epoch 7/25
57/57 [==============================] - 0s 1ms/step - loss: 0.1578 - accuracy: 0.9450
Epoch 8/25
57/57 [==============================] - 0s 1ms/step - loss: 0.1085 - accuracy: 0.9617
Epoch 9/25
57/57 [==============================] - 0s 1ms/step - loss: 0.1060 - accuracy: 0.9589
Epoch 10/25
57/57 [==============================] - 0s 2ms/step - loss: 0.0729 - accuracy: 0.9739
Epoch 11/25
57/57 [

In [26]:
# ask him to perform tricks
model.evaluate(x_test[1:220],  y_test[1:220], verbose=2)

7/7 - 0s - loss: 0.0550 - accuracy: 0.9909


[0.05502455309033394, 0.990867555141449]

In [115]:
# what's the probability?
probability_model = tf.keras.Sequential([
  model,
  tf.keras.layers.Softmax()
])

probability_model(x_test[1:3])

<tf.Tensor: shape=(2, 2), dtype=float32, numpy=
array([[9.9999642e-01, 3.5605894e-06],
       [9.9999213e-01, 7.8312723e-06]], dtype=float32)>

In [116]:
predictNow = model(x_test[120:121]).numpy()
predictNow

array([[-2.6798193, -4.1256447]], dtype=float32)

In [27]:
SMSDataObjNormal_3 = SMSData("/content/sample_data/normal_3HP_6dB_Fan.json", 0)
print('Feature vector B looks like', np.shape(SMSDataObjNormal_3.featVecB))
x_test_a = SMSDataObjNormal_3.featVecB[1:trainIdx, :, :]
y_test_a = SMSDataObjNormal_3.yData[1:trainIdx, :]
print(np.shape(x_test_a), np.shape(y_test_a))                          

Feature vector B looks like (512, 256, 3)
(511, 256, 3) (511, 1)


In [28]:
model.evaluate(x_test_a,  y_test_a, verbose=2)

16/16 - 0s - loss: 0.1175 - accuracy: 0.9589


[0.11750569939613342, 0.9589040875434875]

In [29]:
SMSDataObjabNormal_3 = SMSData("/content/sample_data/abnormal_3HP_6dB_Fan.json", 1)
print('Feature vector B looks like', np.shape(SMSDataObjabNormal_3.featVecB))
x_test_ab = SMSDataObjabNormal_3.featVecB[1:trainIdx, :, :]
y_test_ab = SMSDataObjabNormal_3.yData[1:trainIdx, :]
print(np.shape(x_test_ab), np.shape(y_test_ab))   

Feature vector B looks like (513, 256, 3)
(512, 256, 3) (512, 1)


In [30]:
model.evaluate(x_test_ab,  y_test_ab, verbose=2)

16/16 - 0s - loss: 1.1454 - accuracy: 0.7207


[1.1453884840011597, 0.720703125]